In [3]:
import pandas as pd

# Define file paths for lost list CSVs
file_paths = [
    r"C:\Users\cdub4\OneDrive\Documentos\Seahawks\lost_stnl_and_bp_activity_subset_1.csv",
    r"C:\Users\cdub4\OneDrive\Documentos\Seahawks\lost_stnl_and_bp_activity_subset_2.csv",
    r"C:\Users\cdub4\OneDrive\Documentos\Seahawks\lost_stnl_and_bp_activity_subset_3.csv",
    r"C:\Users\cdub4\OneDrive\Documentos\Seahawks\lost_stnl_and_bp_activity_subset_4.csv",
    r"C:\Users\cdub4\OneDrive\Documentos\Seahawks\lost_stnl_and_bp_activity_subset_5.csv"
]

# Read and combine lost CSV files
lost_dfs = [pd.read_csv(file) for file in file_paths]
combined_lost_df = pd.concat(lost_dfs, ignore_index=True)

# Filter rows where CAMPAIGN_LOST is "STNL to BP" and add a status column
lost_df = combined_lost_df[combined_lost_df['CAMPAIGN_LOST'] == "STNL to BP"].copy()
lost_df['customer_status'] = 'lost'

# Read BP list CSV and add a status column
bp_df = pd.read_csv(r"C:\Users\cdub4\OneDrive\Documentos\Seahawks\bp_activity_subset.csv")
bp_df['customer_status'] = 'current'

# Keep only the columns common to both DataFrames
common_cols = lost_df.columns.intersection(bp_df.columns)
lost_df = lost_df[common_cols]
bp_df = bp_df[common_cols]

# Combine the two DataFrames
combined_customers = pd.concat([lost_df, bp_df], ignore_index=True)

# Optional: Save combined DataFrame to CSV
combined_customers.to_csv(r"C:\Users\cdub4\OneDrive\Documentos\Seahawks\combined_customer_list.csv", index=False)

print(combined_customers.head())


          CUSTOMER_KEY  CUSTOMER_ACTIVITY_KEY LINE_OF_BUSINESS  \
0 -8061130536551660000   -9223361993734550000              NaN   
1 -2653017699136930000   -9223341076556800000              NaN   
2  1013516401290274270   -9223333229117640000              NaN   
3 -7596488359783870000   -9223325886369360000              NaN   
4 -2234331647526360000   -9223324519101500000              NaN   

  ACTIVITY_SOURCE ACTIVITY_CODE  ACTIVITY_TYPE        ACTIVITY_DATETIME  \
0          eloqua    email_open  communication  2017-04-25 18:00:52.357   
1          eloqua    email_send  communication  2024-01-08 00:06:59.290   
2          eloqua    email_send  communication  2020-08-25 21:02:16.683   
3          eloqua    email_send  communication  2017-11-03 14:14:06.690   
4          eloqua    email_open  communication  2021-12-04 11:46:56.013   

   IS_ENGAGEMENT_ACTIVITY customer_status  
0                       1            lost  
1                       0            lost  
2                   

In [5]:
print(combined_customers['customer_status'].value_counts())


customer_status
lost       4006493
current    1116084
Name: count, dtype: int64


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set reference date
reference_date = pd.Timestamp('now')

# Ensure ACTIVITY_DATETIME is datetime
combined_customers['ACTIVITY_DATETIME'] = pd.to_datetime(combined_customers['ACTIVITY_DATETIME'], errors='coerce')

# Derive last engagement date and days since last engagement
engagement_df = combined_customers[combined_customers['IS_ENGAGEMENT_ACTIVITY'] == 1]
last_engagement = engagement_df.groupby('CUSTOMER_KEY')['ACTIVITY_DATETIME'].max().reset_index().rename(columns={'ACTIVITY_DATETIME': 'last_engagement_date'})
last_engagement['days_since_last_engagement'] = (reference_date - last_engagement['last_engagement_date']).dt.days

# Define purchase codes
game_purchase_types = ["primary_purchase_club", "primary_purchase_ga", "primary_purchase_single", "primary_purchase_sth", "primary_purchase_suite"]
resell_purchase_types = ["secondary_purchase"]
merch_purchase_types = ["merch_purchase"]
purchase_codes = game_purchase_types + resell_purchase_types + merch_purchase_types

# Filter purchase events and compute counts
purchase_df = combined_customers[(combined_customers['ACTIVITY_TYPE'] == 'purchase') & (combined_customers['ACTIVITY_CODE'].isin(purchase_codes))]

game_counts = purchase_df[purchase_df['ACTIVITY_CODE'].isin(game_purchase_types)].groupby('CUSTOMER_KEY').size().reset_index(name='game_purchase_count')
merch_counts = purchase_df[purchase_df['ACTIVITY_CODE'].isin(merch_purchase_types)].groupby('CUSTOMER_KEY').size().reset_index(name='merch_purchase_count')
resell_counts = purchase_df[purchase_df['ACTIVITY_CODE'].isin(resell_purchase_types)].groupby('CUSTOMER_KEY').size().reset_index(name='resell_purchase_count')

# Merge counts with last engagement info
customer_summary = last_engagement.copy()
customer_summary = customer_summary.merge(game_counts, on='CUSTOMER_KEY', how='left')
customer_summary = customer_summary.merge(merch_counts, on='CUSTOMER_KEY', how='left')
customer_summary = customer_summary.merge(resell_counts, on='CUSTOMER_KEY', how='left')

# Fill NaN counts with 0
for col in ['game_purchase_count', 'merch_purchase_count', 'resell_purchase_count']:
    customer_summary[col] = customer_summary[col].fillna(0)

# Merge customer_status if available
customer_status = combined_customers[['CUSTOMER_KEY','customer_status']].drop_duplicates()
customer_summary = customer_summary.merge(customer_status, on='CUSTOMER_KEY', how='left')

print(customer_summary.head(100))


           CUSTOMER_KEY    last_engagement_date  days_since_last_engagement  \
0  -9222442665376300000 2024-12-10 18:25:11.160                          54   
1  -9219762487645540000 2023-05-09 19:08:53.440                         635   
2  -9219156172471900000 2022-08-01 12:29:51.188                         917   
3  -9219114118600400000 2025-01-10 18:06:16.247                          23   
4  -9217062962888802692 2024-10-14 07:00:40.000                         112   
..                  ...                     ...                         ...   
95 -9040186546145990000 2022-08-01 12:30:10.830                         917   
96 -9039277127145240000 2024-11-20 21:27:05.103                          74   
97 -9039198197510877527 2024-12-10 14:56:44.390                          54   
98 -9038257846757470000 2024-08-30 18:16:14.561                         156   
99 -9036987208130000000 2023-06-27 14:26:53.685                         586   

    game_purchase_count  merch_purchase_count  rese

In [8]:
print(customer_summary.groupby('customer_status').agg({
    'days_since_last_engagement': ['mean', 'median', 'std'],
    'game_purchase_count': ['mean', 'median', 'std'],
    'merch_purchase_count': ['mean', 'median', 'std'],
    'resell_purchase_count': ['mean', 'median', 'std']
}))


                days_since_last_engagement                     \
                                      mean median         std   
customer_status                                                 
current                         140.704667   89.0  140.746727   
lost                            450.110385  234.0  502.851731   

                game_purchase_count                  merch_purchase_count  \
                               mean median       std                 mean   
customer_status                                                             
current                    1.842000    1.0  2.027752             0.787333   
lost                       0.332057    0.0  1.571728             0.314260   

                                 resell_purchase_count                   
                median       std                  mean median       std  
customer_status                                                          
current            0.0  2.513361              0.569333    0.0  1.4

In [9]:
# Compute correlation matrix
corr_cols = ['days_since_last_engagement', 'game_purchase_count', 'merch_purchase_count', 'resell_purchase_count']
print(customer_summary[corr_cols].corr())

# Prepare data for logistic regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Create a binary target: lost = 1, current = 0
customer_summary['lost_flag'] = (customer_summary['customer_status'] == 'lost').astype(int)

# Fill missing values if necessary
X = customer_summary[corr_cols].fillna(0)
y = customer_summary['lost_flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print("Model coefficients:", dict(zip(corr_cols, model.coef_[0])))


                            days_since_last_engagement  game_purchase_count  \
days_since_last_engagement                    1.000000            -0.201462   
game_purchase_count                          -0.201462             1.000000   
merch_purchase_count                         -0.114527             0.159804   
resell_purchase_count                        -0.131462             0.184114   

                            merch_purchase_count  resell_purchase_count  
days_since_last_engagement             -0.114527              -0.131462  
game_purchase_count                     0.159804               0.184114  
merch_purchase_count                    1.000000               0.126472  
resell_purchase_count                   0.126472               1.000000  
              precision    recall  f1-score   support

           0       0.31      0.07      0.12       448
           1       0.86      0.97      0.91      2666

    accuracy                           0.84      3114
   macro avg    

In [13]:
# Calculate correlations between lost_flag and the four variables
cols = ['days_since_last_engagement', 'game_purchase_count', 'merch_purchase_count', 'resell_purchase_count']
correlations = customer_summary[cols + ['lost_flag']].corr()['lost_flag'][cols]
print(correlations)


days_since_last_engagement    0.226373
game_purchase_count          -0.307113
merch_purchase_count         -0.093855
resell_purchase_count        -0.150126
Name: lost_flag, dtype: float64


In [14]:
# Calculate email count per CUSTOMER_KEY where ACTIVITY_CODE is 'email_send'
email_counts = combined_customers[combined_customers['ACTIVITY_CODE'] == 'email_send'].groupby('CUSTOMER_KEY').size().reset_index(name='email_count')

# Merge the email counts with customer_summary and fill missing values with 0
customer_summary = customer_summary.merge(email_counts, on='CUSTOMER_KEY', how='left')
customer_summary['email_count'] = customer_summary['email_count'].fillna(0)

# Calculate correlations between lost_flag and the 5 variables including email_count
cols = ['days_since_last_engagement', 'game_purchase_count', 'merch_purchase_count', 'resell_purchase_count', 'email_count']
correlations = customer_summary[cols + ['lost_flag']].corr()['lost_flag'][cols]
print(correlations)


days_since_last_engagement    0.226373
game_purchase_count          -0.307113
merch_purchase_count         -0.093855
resell_purchase_count        -0.150126
email_count                  -0.123920
Name: lost_flag, dtype: float64


As seen above, game purchase count is most correlated with customer dropoff from the STNL to BP list

In [16]:

combined_lost_df = pd.concat(lost_dfs, ignore_index=True)

# Filter lost list for CAMPAIGN_LOST == "BP to STH" and add a status column
lost_df = combined_lost_df[combined_lost_df['CAMPAIGN_LOST'] == "BP to STH"].copy()
lost_df['customer_status'] = 'lost'

# Read current STH dataset
sth_df = pd.read_csv(r"C:/Users/cdub4/OneDrive/Documentos/Seahawks/new_sth_subset.csv")
sth_df['customer_status'] = 'current'

# Keep only common columns and combine the datasets
common_cols = lost_df.columns.intersection(sth_df.columns)
lost_df = lost_df[common_cols]
sth_df = sth_df[common_cols]
combined_customers = pd.concat([lost_df, sth_df], ignore_index=True)

# Ensure ACTIVITY_DATETIME is datetime
combined_customers['ACTIVITY_DATETIME'] = pd.to_datetime(combined_customers['ACTIVITY_DATETIME'], errors='coerce')
reference_date = pd.Timestamp('now')

# Derive last engagement date and compute days_since_last_engagement
engagement_df = combined_customers[combined_customers['IS_ENGAGEMENT_ACTIVITY'] == 1]
last_engagement = engagement_df.groupby('CUSTOMER_KEY')['ACTIVITY_DATETIME'].max().reset_index().rename(columns={'ACTIVITY_DATETIME': 'last_engagement_date'})
last_engagement['days_since_last_engagement'] = (reference_date - last_engagement['last_engagement_date']).dt.days

# Define purchase codes
game_purchase_types = ["primary_purchase_club", "primary_purchase_ga", "primary_purchase_single", "primary_purchase_sth", "primary_purchase_suite"]
resell_purchase_types = ["secondary_purchase"]
merch_purchase_types = ["merch_purchase"]
purchase_codes = game_purchase_types + resell_purchase_types + merch_purchase_types

# Filter purchase events and compute purchase counts
purchase_df = combined_customers[(combined_customers['ACTIVITY_TYPE'] == 'purchase') & (combined_customers['ACTIVITY_CODE'].isin(purchase_codes))]
game_counts = purchase_df[purchase_df['ACTIVITY_CODE'].isin(game_purchase_types)].groupby('CUSTOMER_KEY').size().reset_index(name='game_purchase_count')
merch_counts = purchase_df[purchase_df['ACTIVITY_CODE'].isin(merch_purchase_types)].groupby('CUSTOMER_KEY').size().reset_index(name='merch_purchase_count')
resell_counts = purchase_df[purchase_df['ACTIVITY_CODE'].isin(resell_purchase_types)].groupby('CUSTOMER_KEY').size().reset_index(name='resell_purchase_count')

# Calculate email counts
email_counts = combined_customers[combined_customers['ACTIVITY_CODE'] == 'email_send'].groupby('CUSTOMER_KEY').size().reset_index(name='email_count')

# Merge all metrics into a customer_summary
customer_summary = last_engagement.copy()
customer_summary = customer_summary.merge(game_counts, on='CUSTOMER_KEY', how='left')
customer_summary = customer_summary.merge(merch_counts, on='CUSTOMER_KEY', how='left')
customer_summary = customer_summary.merge(resell_counts, on='CUSTOMER_KEY', how='left')
customer_summary = customer_summary.merge(email_counts, on='CUSTOMER_KEY', how='left')

# Fill missing counts with 0
for col in ['game_purchase_count', 'merch_purchase_count', 'resell_purchase_count', 'email_count']:
    customer_summary[col] = customer_summary[col].fillna(0)

# Merge customer_status
status_df = combined_customers[['CUSTOMER_KEY', 'customer_status']].drop_duplicates()
customer_summary = customer_summary.merge(status_df, on='CUSTOMER_KEY', how='left')

# Create binary flag for lost (lost = 1, current = 0)
customer_summary['lost_flag'] = (customer_summary['customer_status'] == 'lost').astype(int)

# Compute correlations between lost_flag and selected variables
cols = ['days_since_last_engagement', 'game_purchase_count', 'merch_purchase_count', 'resell_purchase_count', 'email_count']
correlations = customer_summary[cols + ['lost_flag']].corr()['lost_flag'][cols]
print(correlations)

days_since_last_engagement    0.312465
game_purchase_count          -0.179496
merch_purchase_count         -0.040606
resell_purchase_count        -0.020248
email_count                  -0.022551
Name: lost_flag, dtype: float64


As seen above, days since last engagement has the highest correlation with customer dropoff from the BP to STH list.